In [1]:
import time
import random
import re
import os
import csv
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime, timedelta

import urllib.request # URL requests
import urllib.parse # URL parsing

import requests # HTTP requests
from bs4 import BeautifulSoup as bs # HTML parsing

from selenium import webdriver # Chromedriver
from selenium.webdriver.common.by import By # HTML element 선택
from selenium.webdriver.common.keys import Keys # 키보드 키 입력 위함
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities # 크롬 옵션 설정
from selenium.common.exceptions import UnexpectedAlertPresentException # 예외 처리 위함
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException

In [2]:
driver = webdriver.Chrome() # 크롬드라이버 객체 생성(초기화)
driver.implicitly_wait(3) # 암묵적 대기(3초)

In [3]:
# 키워드 입력
keyword = ['건강']

In [4]:
# url 폴더 생성
if not os.path.exists('url'):
    os.makedirs('url')

In [5]:
for word in keyword:
    extracted_data = []
    
    driver.get(f'https://www.clien.net/service/search?q={word}')
    
    # 정확도순 버튼 클릭
    button = driver.find_element(By.XPATH, '//*[@id="div_content"]/div[2]/div[2]/a[2]')
    button.click()

    # 첫 번째 버튼 클릭 (1->2페이지)
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, ".list_item.symph_row.jirum")

        for element in elements:
            try:
                href_element = element.find_element(By.CSS_SELECTOR, "a.subject_fixed")
                href = href_element.get_attribute("href")
                
                timestamp_element = element.find_element(By.CSS_SELECTOR, "span.timestamp")
                timestamp_text = timestamp_element.get_attribute("textContent").strip()

                extracted_data.append({"href": href, "timestamp": timestamp_text})

            except Exception as e:
                print(f"An error occurred: {e}")
        
        first_button = driver.find_element(By.XPATH, '//*[@id="div_content"]/div[7]/div/a[1]')
        first_button.click()
        time.sleep(1)
    except Exception as e:
        print(f"예외 발생: {e}")

    # (2->3->...->11페이지) 버튼 클릭
    for i in range(9):
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, ".list_item.symph_row.jirum")
            for element in elements:
                try:
                    href_element = element.find_element(By.CSS_SELECTOR, "a.subject_fixed")
                    href = href_element.get_attribute("href")
                    
                    timestamp_element = element.find_element(By.CSS_SELECTOR, "span.timestamp")
                    timestamp_text = timestamp_element.get_attribute("textContent").strip()
                    
                    extracted_data.append({"href": href, "timestamp": timestamp_text})
                except Exception as e:
                    print(f"데이터 수집 중 예외 발생: {e}")

            # 두 번째 버튼을 클릭하여 다음 페이지로 이동
            second_button = driver.find_element(By.XPATH, '//*[@id="div_content"]/div[7]/div/a[3]')
            second_button.click()
            time.sleep(1)
        except Exception as e:
            print(f"페이지 이동 중 예외 발생: {e}")
            break

    # (11->12->...->40페이지) 버튼 클릭
    for i in range(30):
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, ".list_item.symph_row.jirum")
            for element in elements:
                try:
                    href_element = element.find_element(By.CSS_SELECTOR, "a.subject_fixed")
                    href = href_element.get_attribute("href")
                    
                    timestamp_element = element.find_element(By.CSS_SELECTOR, "span.timestamp")
                    timestamp_text = timestamp_element.get_attribute("textContent").strip()
                    
                    extracted_data.append({"href": href, "timestamp": timestamp_text})
                except Exception as e:
                    print(f"데이터 수집 중 예외 발생: {e}")

            # 세 번째 버튼을 클릭하여 다음 페이지로 이동
            third_button = driver.find_element(By.XPATH, '//*[@id="div_content"]/div[7]/div/a[4]')
            third_button.click()
            time.sleep(1)
        except Exception as e:
            print(f"페이지 이동 중 예외 발생: {e}")
            break

    # (40->41->...->50페이지) 버튼 클릭
    for i in range(1, 11):
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, ".list_item.symph_row.jirum")
            for element in elements:
                try:
                    href_element = element.find_element(By.CSS_SELECTOR, "a.subject_fixed")
                    href = href_element.get_attribute("href")
                    
                    timestamp_element = element.find_element(By.CSS_SELECTOR, "span.timestamp")
                    timestamp_text = timestamp_element.get_attribute("textContent").strip()
                    
                    extracted_data.append({"href": href, "timestamp": timestamp_text})
                except Exception as e:
                    print(f"데이터 수집 중 예외 발생: {e}")

            # 다음 버튼 클릭하여 페이지 이동
            fourth_button = driver.find_element(By.XPATH, f'//*[@id="div_content"]/div[7]/div/div/a[{i}]')
            fourth_button.click()
            time.sleep(1)
        except Exception as e:
            print(f"페이지 이동 중 예외 발생: {e}")
            break

    # 데이터 확인
    extracted_data = pd.DataFrame(extracted_data)
    extracted_data.to_csv(f"url/{word}.csv", index=False)